# Load imports and create config directory

In [ ]:
# __main__.py
import argparse
import os

from loguru import logger

from mkv_episode_matcher.config import get_config, set_config
from mkv_episode_matcher.utils import check_filename, cleanup_ocr_files, get_subtitles
from mkv_episode_matcher.tmdb_client import fetch_show_id,fetch_season_details
import os
import re
import shutil
from typing import Set

import requests
from loguru import logger
from opensubtitlescom import OpenSubtitles
# Log the start of the application
logger.info("Starting the application")



# Check if the configuration directory exists, if not create it
if not os.path.exists(os.path.join(os.path.expanduser("~"), ".mkv-episode-matcher")):
    os.makedirs(os.path.join(os.path.expanduser("~"), ".mkv-episode-matcher"))

# Define the paths for the configuration file and cache directory
CONFIG_FILE = os.path.join(
    os.path.expanduser("~"), ".mkv-episode-matcher", "config.ini"
)
CACHE_DIR = os.path.join(os.path.expanduser("~"), ".mkv-episode-matcher", "cache")

# Load configuration settings from config.ini

In [ ]:
config = get_config(CONFIG_FILE)
show_dir = config.get("show_dir")
show_name = os.path.basename(show_dir)
series_name = os.path.basename(show_dir)
tmdb_api_key = config.get("tmdb_api_key")
open_subtitles_api_key = config.get("open_subtitles_api_key")
open_subtitles_user_agent = config.get("open_subtitles_user_agent")
open_subtitles_username = config.get("open_subtitles_username")
open_subtitles_password = config.get("open_subtitles_password")

# Make sure all required info exists in config.ini

In [ ]:
for x in     [
        show_dir,
        tmdb_api_key,
        open_subtitles_api_key,
        open_subtitles_user_agent,
        open_subtitles_username,
        open_subtitles_password,
    ]:
    try:
        print(x)
    except:
        print('failed')

In [ ]:
if not all(
    [
        show_dir,
        tmdb_api_key,
        open_subtitles_api_key,
        open_subtitles_user_agent,
        open_subtitles_username,
        open_subtitles_password,
    ]
):
    logger.error("Missing configuration settings. Please run the setup script.")

# Make sure show can be found on TMDb
The show id is used to search on opensubtitles

In [ ]:
show_id = fetch_show_id(show_name)
if show_id is None:
    logger.error(f"Could not find show '{os.path.basename(show_name)}' on TMDb.")
else:
    print(show_id)

# Try getting the first season automatically

In [ ]:
get_subtitles(show_id, seasons=set([1]))

# Check if there's an issue in the get_subtitles function

In [ ]:
try:
    # Initialize the OpenSubtitles client
    subtitles = OpenSubtitles(open_subtitles_user_agent, open_subtitles_api_key)

    # Log in (retrieve auth token)
    subtitles.login(open_subtitles_username, open_subtitles_password)
except Exception as e:
    logger.error(f"Failed to log in to OpenSubtitles: {e}")

for season in [1]:
    episodes = fetch_season_details(show_id, season)
    logger.info(f"Found {episodes} episodes in Season {season}")

    for episode in range(1, episodes + 1):
        logger.info(f"Processing Season {season}, Episode {episode}...")
        series_cache_dir =os.path.join(
            CACHE_DIR,
            "data",
            series_name)
        os.makedirs(series_cache_dir,exist_ok=True)
        srt_filepath = os.path.join(
            series_cache_dir,
            f"{series_name} - S{season:02d}E{episode:02d}.srt",
        )
        if not os.path.exists(srt_filepath):
            # get the episode info from TMDB
            url = f"https://api.themoviedb.org/3/tv/{show_id}/season/{season}/episode/{episode}?api_key={tmdb_api_key}"
            response = requests.get(url)
            response.raise_for_status()
            episode_data = response.json()
            episode_name = episode_data["name"]
            episode_id = episode_data["id"]
            # search for the subtitle
            response = subtitles.search(tmdb_id=episode_id, languages="en")
            if len(response.data) == 0:
                logger.warning(
                    f"No subtitles found for {series_name} - S{season:02d}E{episode:02d}"
                )

            for subtitle in response.data:
                subtitle_dict = subtitle.to_dict()
                # Remove special characters and convert to uppercase
                filename_clean = re.sub(
                    r"\W+", " ", subtitle_dict["file_name"]
                ).upper()
                if f"E{episode:02d}" in filename_clean:
                    logger.info(f"Original filename: {subtitle_dict['file_name']}")
                    srt_file = subtitles.download_and_save(subtitle)
                    series_name = series_name.replace(":", " -")
                    shutil.move(os.path.join(os.getcwd(),srt_file), srt_filepath)
                    logger.info(f"Subtitle saved to {srt_filepath}")
                    break
                else:
                    continue
        else:
            logger.info(
                f"Subtitle already exists for {series_name} - S{season:02d}E{episode:02d}"
            )
            continue